In [8]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import Callback
import tensorflow_datasets as tfds
(dataset, dataset_info) = tfds.load('cats_vs_dogs', split='train[:90%]', with_info=True, as_supervised=True)
num_classes = dataset_info.features['label'].num_classes
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.2, upper=1.8)
    image = tf.image.random_hue(image, max_delta=0.1)
    image = tf.image.random_saturation(image, lower=0.2, upper=1.8)
    image = tf.image.per_image_standardization(image)
    label = tf.one_hot(label, num_classes)
    return image, label
dataset = dataset.map(preprocess_image)
num_samples = dataset_info.splits['train[:90%]'].num_examples
num_train_samples = int(0.8 * num_samples)
train_dataset = dataset.take(num_train_samples)
val_dataset = dataset.skip(num_train_samples)
batch_size = 32
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)
class TrainingAccuracyCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch + 1}/{self.params['epochs']} - Training Accuracy: {logs['accuracy']:.4f}")
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    #Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
accuracy_callback = TrainingAccuracyCallback()
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset, callbacks=[accuracy_callback])
final_training_accuracy = history.history['accuracy'][-1]
print("Training accuracy:", final_training_accuracy)


Epoch 1/10
524/524 [==============================] - 134s 238ms/step - loss: 0.7487 - accuracy: 0.5316 - val_loss: 0.6938 - val_accuracy: 0.4878
Epoch 2/10
524/524 [==============================] - 140s 267ms/step - loss: 0.6939 - accuracy: 0.5069 - val_loss: 0.6937 - val_accuracy: 0.4878
Epoch 3/10
524/524 [==============================] - 141s 268ms/step - loss: 0.6962 - accuracy: 0.5125 - val_loss: 0.6940 - val_accuracy: 0.4878
Epoch 4/10
524/524 [==============================] - 121s 231ms/step - loss: 0.6940 - accuracy: 0.5067 - val_loss: 0.6933 - val_accuracy: 0.4878
Epoch 5/10
524/524 [==============================] - 140s 267ms/step - loss: 0.6941 - accuracy: 0.5120 - val_loss: 0.6943 - val_accuracy: 0.4878
Epoch 6/10
524/524 [==============================] - 120s 229ms/step - loss: 0.6738 - accuracy: 0.5720 - val_loss: 0.6193 - val_accuracy: 0.6394
Epoch 7/10
524/524 [==============================] - 120s 228ms/step - loss: 0.5706 - accuracy: 0.7006 - val_loss: 0.5009 -

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
